Pytorch Resnet ：https://www.kaggle.com/a763337092/blending-tensorflow-and-pytorch


TensorFlow-Resnet-Inference：https://www.kaggle.com/quincyqiang/tensorflow-resnet-inference?rvi=1

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa 
import gc
tf.__version__

In [ ]:
# 种子设置
SEED = 1111
tf.random.set_seed(SEED)
np.random.seed(SEED)


In [ ]:
%%time
# train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train=pd.read_parquet('../input/step01-csv-parquet/dtrain.parquet')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
train['action'] = ((train['resp'].values) > 0).astype(int)

train.fillna(train.mean(),inplace=True)

# 测试集所需的填充缺失值 f_mean
raw_features = [c for c in train.columns if "feature" in c]
f_mean = np.mean(train[raw_features[1:]].values,axis=0)

np.save("f_mean_online.npy",f_mean)
train.shape

In [ ]:
def stats_features(tmp_df):
    # 构建两个交叉特征：https://www.kaggle.com/a763337092/pytorch-resnet-starter-inference
    tmp_df['feature_cross_41_42_43']=tmp_df['feature_41']+tmp_df['feature_42']+tmp_df['feature_43']
    tmp_df['feature_cross_1_2']=tmp_df['feature_1']/(tmp_df['feature_2']+1e-5)

    # 行：sum mean std median
#     tmp_df['features_sum']=tmp_df[raw_features].sum(axis=1)
#     tmp_df['features_mean']=tmp_df[raw_features].mean(axis=1)
#     tmp_df['features_std']=tmp_df[raw_features].std(axis=1)
#     tmp_df['features_var']=tmp_df[raw_features].var(axis=1)
#     tmp_df['features_median']=tmp_df[raw_features].median(axis=1)

    # skew sem kurtosis kurt
    #tmp_df['features_skew']=tmp_df[raw_features].skew(axis=1)
    #tmp_df['features_kurtosis']=tmp_df[raw_features].kurtosis(axis=1)
    #tmp_df['features_kurt']=tmp_df[raw_features].kurt(axis=1) # Wall time: 23.4 s

    #tmp_df['features_sem']=tmp_df[raw_features].sem(axis=1) # Wall time: 18.3 s

    # 行：max min  
#     tmp_df['features_max']=tmp_df[raw_features].max(axis=1)
#     tmp_df['features_min']=tmp_df[raw_features].min(axis=1)
    # print(train[['features_sum','features_mean','features_std','features_median','features_max','features_min']].head())
    # pct
    # train['features_prod']=train[raw_features].prod(axis=1)
    # train['features_prod']

    tmp_df.head()
    return tmp_df
    


In [ ]:
train=stats_features(train)
train.head()

In [ ]:
features = [c for c in train.columns if "feature" in c]

In [ ]:

valid = train.loc[(train.date >= 450) & (train.date < 500)].reset_index(drop=True)
# train = train.loc[train.date < 450].reset_index(drop=True)


resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

X_valid = valid.loc[:, valid.columns.str.contains('feature')]
y_valid = np.stack([(valid[c] > 0).astype('int') for c in resp_cols]).T

del train

In [ ]:
def create_resnet(
    num_columns, num_labels, hidden_size, dropout_rate, label_smoothing, learning_rate
):
    
    

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x=tf.keras.layers.BatchNormalization()(inp)
    x=tf.keras.layers.Dropout(dropout_rate)(x)
    
    
    x1=tf.keras.layers.Dense(hidden_size)(x)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.LeakyReLU()(x1)
    x1=tf.keras.layers.Dropout(dropout_rate)(x1)
    
    x = tf.keras.layers.concatenate([x, x1], axis=1)

    
    x2=tf.keras.layers.Dense(hidden_size)(x)
    x2=tf.keras.layers.BatchNormalization(axis=1)(x2)
    x2=tf.keras.layers.LeakyReLU()(x2)
    x2=tf.keras.layers.Dropout(dropout_rate)(x2)
    
    x = tf.keras.layers.concatenate([x1, x2], axis=1)

    
    x3=tf.keras.layers.Dense(hidden_size)(x)
    x3=tf.keras.layers.BatchNormalization(axis=1)(x3)
    x3=tf.keras.layers.LeakyReLU()(x3)
    x3=tf.keras.layers.Dropout(dropout_rate)(x3)
    
    x = tf.keras.layers.concatenate([x2, x3], axis=1)

    x4=tf.keras.layers.Dense(hidden_size)(x)
    x4=tf.keras.layers.BatchNormalization(axis=1)(x4)
    x4=tf.keras.layers.LeakyReLU()(x4)
    x4=tf.keras.layers.Dropout(dropout_rate)(x4)
    
    x = tf.keras.layers.concatenate([x3, x4], axis=1)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    
    
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#         optimizer=tfa.optimizers.AdamW(learning_rate=learning_rate,weight_decay=1e-5),
        optimizer=tfa.optimizers.RectifiedAdam(lr=learning_rate,weight_decay=1e-5),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model


NUM_FOLDS=5

TRAINING=True
if TRAINING:
    for i in range(NUM_FOLDS):
        tf.keras.backend.clear_session()


        # 种子设置
        SEED = 1111
        tf.random.set_seed(SEED+i)
        np.random.seed(SEED+i)

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                         verbose=1,
                                                         factor=0.2,
                                                         patience=8, mode='min')


        earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath='online_model_{}.weights'.format(i),
            save_weights_only=True,
            monitor='val_AUC',
            mode='max',
            verbose=1,
            save_best_only=True)

        dropout_rate=0.2
        hidden_size=256
        
        label_smoothing = 0.005
        learning_rate = 1e-3

        clf = create_resnet(
            len(features), 5, hidden_size, dropout_rate, label_smoothing, learning_rate
            )

        clf.summary()

        clf.fit(X_train, y_train, epochs=200, batch_size=4096,validation_data=(X_valid,y_valid),
               callbacks=[reduce_lr,
                          earlystop_callback,
                          model_checkpoint_callback])
#         del clf
        gc.collect()


In [ ]:
# from tqdm import tqdm

# th = 0.502

# f = np.median
# clf.call = tf.function(clf.call, experimental_relax_shapes=True)

# models=[clf,clf,clf,clf,clf]
# import janestreet

# env = janestreet.make_env()

# for (test_df, pred_df) in tqdm(env.iter_test()):
#     if test_df['weight'].item() > 0:
# #         x_tt = test_df.loc[:, features].values
# #         if np.isnan(x_tt[:, 1:].sum()):
# #             x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean

#         test_df = stats_features(test_df)
#         x_tt = test_df.loc[:, features].values
#         if np.isnan(x_tt[:, 1:130].sum()):
#             x_tt[:, 1:130] = np.nan_to_num(x_tt[:, 1:130]) + np.isnan(x_tt[:, 1:130]) * f_mean

#         pred = np.mean([model(x_tt, training=False).numpy() for model in models], axis=0)
#         pred = f(pred)
#         pred_df.action = np.where(pred >= th, 1, 0).astype(int)
#     else:
#         pred_df.action = 0

#     env.predict(pred_df)

In [ ]:
nan

In [ ]:
nan

In [ ]:
nan